# Installs

In [ ]:
!pip install pip==21
!pip install tensorboardX
!pip install pyglet==1.5.1
!pip install torchsummary
!pip install optuna
!pip install optuna-dashboard
!pip install torchrl
!pip install setuptools==65.5.1
!pip install gym==0.21.0
!pip install nes-py
!pip install stable-baselines3[extra]
!pip install lz4
!sudo apt-get install -y xvfb
!pip install pyvirtualdisplay

# Imports

In [1]:
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1024, 768))
virtual_display.start()

In [2]:
import sys
import os

# Get the absolute path to the parent directory of gym-tetris
gym_tetris_parent_path = os.path.abspath(os.path.join('..', 'tetris-environments', 'gym-tetris'))

# Append the path to the sys.path
sys.path.append(gym_tetris_parent_path)

In [3]:
import random
import time
from distutils.util import strtobool

from gym import Wrapper, ObservationWrapper
from gym.wrappers import RecordEpisodeStatistics, RecordVideo, FrameStack
from gym.spaces import Box, Discrete

from nes_py.wrappers import JoypadSpace
from gym_tetris.actions import SIMPLE_MOVEMENT
from gym_tetris.tetris_env import TetrisEnv

import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

# from stable_baselines3.common.buffers import ReplayBuffer

from tensordict import TensorDict
from torchrl.data import TensorDictPrioritizedReplayBuffer, LazyTensorStorage
# from torchrl.data import PrioritizedReplayBuffer, ListStorage, LazyMemmapStorage

from torchsummary import summary
from collections import deque

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.1.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [4]:
import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
from optuna.visualization import plot_optimization_history, plot_param_importances

# Model

This sections contains 6 different model architechtures:   
Five models that have been pre-trained using imitation learning, and one "blank" fully-connected model, which serves as a control group

In [5]:
class ImitationNetwork1(nn.Module):
    def __init__(self, num_actions, input_dim=(20, 10)):
        super().__init__()
        self._frames = 4
        self._num_actions = num_actions

        # CNN modeled off of Mnih et al.
        self.cnn = nn.Sequential(
            nn.Conv2d(self._frames, 32, kernel_size=4, stride=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=2, stride=1),
            nn.ReLU()
        )

        self.fc_layer_inputs = self.cnn_out_dim(input_dim)

        self.fully_connected = nn.Sequential(
            nn.Linear(self.fc_layer_inputs, 512, bias=True),
            nn.ReLU(),
            nn.Linear(512, 128, bias=True),
            nn.ReLU(),
            nn.Linear(128, self._num_actions))
        
        # Load existing state dict
        state_dict = torch.load("Imitation_Models/1-active-4stacked-v1-84")
        self.load_state_dict(state_dict)

    def cnn_out_dim(self, input_dim):
        return self.cnn(torch.zeros(1, self._frames, *input_dim)).flatten().shape[0]

    def forward(self, x):
        cnn_out = self.cnn(x).reshape(-1, self.fc_layer_inputs)
        return self.fully_connected(cnn_out)

In [6]:
class ImitationNetwork2(nn.Module):
    def __init__(self, num_actions, input_dim=(20, 10)):
        super().__init__()
        self._frames = 4
        self._num_actions = num_actions

        # CNN modeled off of Mnih et al.
        self.cnn = nn.Sequential(
            nn.Conv2d(self._frames, 32, kernel_size=4, stride=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=2, stride=1),
            nn.ReLU()
        )

        self.fc_layer_inputs = self.cnn_out_dim(input_dim)

        self.fully_connected = nn.Sequential(
            nn.Linear(self.fc_layer_inputs, 512, bias=True),
            nn.ReLU(),
            nn.Linear(512, 128, bias=True),
            nn.ReLU(),
            nn.Linear(128, self._num_actions))
        
        # Load existing state dict
        state_dict = torch.load("Imitation_Models/1-active-4stacked-v1-90")
        self.load_state_dict(state_dict)

    def cnn_out_dim(self, input_dim):
        return self.cnn(torch.zeros(1, self._frames, *input_dim)).flatten().shape[0]

    def forward(self, x):
        cnn_out = self.cnn(x).reshape(-1, self.fc_layer_inputs)
        return self.fully_connected(cnn_out)

In [7]:
class ImitationNetwork3(nn.Module):
    def __init__(self, num_actions, input_dim=(20, 10)):
        super().__init__()
        self._frames = 4
        self._num_actions = num_actions

        # CNN modeled off of Mnih et al.
        self.cnn = nn.Sequential(
            nn.Conv2d(self._frames, 32, kernel_size=4, stride=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=2, stride=1),
            nn.ReLU()
        )

        self.fc_layer_inputs = self.cnn_out_dim(input_dim)

        self.fully_connected = nn.Sequential(
            nn.Linear(self.fc_layer_inputs, 512, bias=True),
            nn.ReLU(),
            nn.Linear(512, 128, bias=True),
            nn.ReLU(),
            nn.Linear(128, self._num_actions))
        
        # Load existing state dict
        state_dict = torch.load("Imitation_Models/5-active-4stacked-v1-102")
        self.load_state_dict(state_dict)

    def cnn_out_dim(self, input_dim):
        return self.cnn(torch.zeros(1, self._frames, *input_dim)).flatten().shape[0]

    def forward(self, x):
        cnn_out = self.cnn(x).reshape(-1, self.fc_layer_inputs)
        return self.fully_connected(cnn_out)

In [8]:
class ImitationNetwork4(nn.Module):
    def __init__(self, num_actions, input_dim=(20, 10)):
        super().__init__()
        self._frames = 4
        self.num_actions = num_actions

        # CNN modeled off of Mnih et al.
        self.cnn = nn.Sequential(
            nn.Conv2d(self._frames, 32, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=2, stride=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=2, stride=1),
            nn.ReLU()
        )

        self.fc_layer_inputs = self.cnn_out_dim(input_dim)

        self.fully_connected = nn.Sequential(
            nn.Linear(self.fc_layer_inputs, 512, bias=True),
            nn.ReLU(),
            nn.Linear(512, self.num_actions))
        
        # Load existing state dict
        state_dict = torch.load("Imitation_Models/8-active-4stacked-v2-79")
        self.load_state_dict(state_dict)
        

    def cnn_out_dim(self, input_dim):
        return self.cnn(torch.zeros(1, self._frames, *input_dim)).flatten().shape[0]

    def forward(self, x):
        cnn_out = self.cnn(x).reshape(-1, self.fc_layer_inputs)
        return self.fully_connected(cnn_out)

In [9]:
class ImitationNetwork5(nn.Module):
    def __init__(self, num_actions, input_dim=(20,10)):
        super().__init__()
        self._frames = 4
        self._num_actions = num_actions

        # CNN modeled off of Mnih et al.
        self.cnn = nn.Sequential(
            nn.Conv2d(self._frames, 32, kernel_size=4, stride=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=2, stride=1),
            nn.ReLU(),
        )

        self.fc_layer_inputs = self.cnn_out_dim(input_dim)

        self.fully_connected = nn.Sequential(
            nn.Linear(self.fc_layer_inputs, 512, bias=True),
            nn.ReLU(),
            nn.Linear(512, self._num_actions))
        
        # Load existing state dict
        state_dict = torch.load("Imitation_Models/8-active-4stacked-v4-78")
        self.load_state_dict(state_dict)

    def cnn_out_dim(self, input_dim):
        return self.cnn(torch.zeros(1, self._frames, *input_dim)).flatten().shape[0]

    def forward(self, x):
        cnn_out = self.cnn(x).reshape(-1, self.fc_layer_inputs)
        return self.fully_connected(cnn_out)

In [10]:
# FC network
class FC_QNetwork(nn.Module):
    def __init__(self, num_actions):
        super().__init__()
        self._network = nn.Sequential(
            # (4, 20, 10)
            nn.Flatten(),
            # 800
            nn.Linear(800, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, num_actions),
        )
    
    def forward(self, x):
        return self._network(x)

In [11]:
def get_model_class(model_name : str):
    if model_name == "FC":
        return FC_QNetwork
    elif model_name == "Imitation1":
        return ImitationNetwork1
    elif model_name == "Imitation2":
        return ImitationNetwork2
    elif model_name == "Imitation3":
        return ImitationNetwork3
    elif model_name == "Imitation4":
        return ImitationNetwork4
    elif model_name == "Imitation5":
        return ImitationNetwork5
    else:
        print("Not a valid architecture")

# Environment

In [12]:
# DEVICE_NAME = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE_NAME = "cpu"
DEVICE = torch.device(DEVICE_NAME)

In [13]:
# Frame Skip
class FrameSkipEnv(Wrapper):
    def __init__(self, env=None, skip=4):
        super(FrameSkipEnv, self).__init__(env)
        self._skip = skip

    def step(self, action):
        total_reward = 0.0
        done = None
        for _ in range(self._skip):
            obs, reward, done, info = self.env.step(action)
            total_reward += reward
            if done:
                break
        return obs, total_reward, done, info

    def reset(self):
        obs = self.env.reset()
        return obs

In [14]:
# Board Constants
GAME_BOX = 47, 95, 209, 176
BOARD_SHAPE = 20, 10
y_step = (GAME_BOX[2] - GAME_BOX[0]) // BOARD_SHAPE[0]
x_step = (GAME_BOX[3] - GAME_BOX[1]) // BOARD_SHAPE[1]

In [15]:
# Binary Board
class BinaryBoard(ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
        self.observation_space = Box(0, 1, BOARD_SHAPE)

    def observation(self, obs):
        # Given an image of the current board, obtain a binary (20x10) representation
        gray = np.mean(obs, axis=-1)
        cropped = gray[GAME_BOX[0]+(y_step//2) : GAME_BOX[2] : y_step,
                       GAME_BOX[1]+(x_step//2) : GAME_BOX[3] : x_step]
        assert cropped.shape == BOARD_SHAPE
        cropped[cropped > 1] = 1
        return cropped

In [16]:
# Tensor Wrapper
class TensorWrapper(ObservationWrapper):
    def __init__(self, env=None):
        super(TensorWrapper, self).__init__(env)
    
    def observation(self, obs):
        np_obs = np.array([obs])
        return torch.Tensor(np_obs).to(DEVICE)

In [17]:
# Get Environment
FRAME_SKIP = 6
# Making an environment
def get_env(args, run_name : str = "run", capture_video : bool = False):
    env = TetrisEnv(
        line_weight=args.line_weight,
        height_weight=args.height_weight,
        cost_weight=args.cost_weight,
        holes_weight=args.holes_weight,
        bumpiness_weight=args.bumpiness_weight,
        col_transitions_weight=args.col_transitions_weight,
        row_transitions_weight=args.row_transitions_weight,
    )

    env = RecordEpisodeStatistics(env)
    if capture_video:
        env = RecordVideo(env, f"videos/{run_name}", episode_trigger=lambda ep_num: ep_num % args.video_frequency == 0)
    
    env = JoypadSpace(env, args.movement)
    env = FrameSkipEnv(env, skip=FRAME_SKIP)
    env = BinaryBoard(env)
    env = FrameStack(env, args.frame_stack)
    env = TensorWrapper(env)

    env.seed(args.seed)
    env.action_space.seed(args.seed)
    env.observation_space.seed(args.seed)
    return env

# Training

In [18]:
# Epsilon scheduling
def linear_schedule(start_e: float, end_e: float, duration: int, t: int):
    slope = (end_e - start_e) / duration
    return max(slope * t + start_e, end_e)

In [19]:
# Video Constants
FPS = 60 / FRAME_SKIP
SCALE_UP = 10

In [20]:
# Write the current observation into the video
def write_video_frame(out, obs, frame_stack : int = 1, scale_up : int = 1):
    print("Writing Model Inputs Video Frame. Should not happen right now!")
    img = np.array(obs.cpu(), dtype='uint8')[0] * 255
    if frame_stack > 1:
        img = img[-1]
    if scale_up > 1:
        img = np.repeat(np.repeat(img, scale_up, axis=0), scale_up, axis=1)
    out.write(img)

In [21]:
# Evaluation
def evaluate(args, model: torch.nn.Module, eval_name : str = "eval", no_video : bool = False):
    env = get_env(args, run_name=eval_name, capture_video=args.capture_eval_video) 
    
    model.eval()

    total_lines = 0.0
    for episode in range(args.eval_episodes):
        if args.capture_inputs_video and not no_video:
            out = cv2.VideoWriter(f'eval_episode{episode}.mp4', cv2.VideoWriter_fourcc(*'mp4v'), FPS, (BOARD_SHAPE[1]*SCALE_UP, BOARD_SHAPE[0]*SCALE_UP), False)
        
        obs = env.reset()
        done = False
        while not done:
            if args.capture_inputs_video and not no_video:
                write_video_frame(out, obs, args.frame_stack, SCALE_UP)

            q_values = model(obs)
            action = int(torch.argmax(q_values))
            obs, _, done, info = env.step(action)
        
        total_lines += info.get('lines')
    env.close()
    mean_lines = total_lines / args.eval_episodes
    return mean_lines

In [22]:
def write_episode_scalars(writer, global_step, info, epsilon):
    writer.add_scalar("charts/episodic_return", info.get("episode")["r"], global_step)
    writer.add_scalar("charts/episodic_length", info.get("episode")["l"], global_step)
    writer.add_scalar("charts/epsilon", epsilon, global_step)
    # writer.add_scalar("charts/score", info.get("score"), global_step)
    writer.add_scalar("charts/lines", info.get("lines"), global_step)

In [23]:
# Single env training without optuna - for simplicity
def train(args, start_model_path=None, trial=None):
  try:
    run_name = f"{args.exp_name}__{args.seed}__{args.run_id}"
    prefix = ""

    if trial:
      run_name += f"_trial_{trial.number}"
      prefix = f"trial_{trial.number}: "

    writer = SummaryWriter(f"runs/{run_name}")
    writer.add_text(
        "hyperparameters",
        "|param|value|\n|-|-|\n%s" % ("\n".join([f"|{key}|{value}|" for key, value in vars(args).items()])),
    )

    # Random seeding
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    torch.backends.cudnn.deterministic = args.torch_deterministic

    # env setup
    env = get_env(args, run_name=f"{run_name}_0", capture_video=args.capture_video)
    assert isinstance(env.action_space, Discrete), "only discrete action space is supported"

    # model setup
    q_network = args.model(env.action_space.n).to(DEVICE)
    if start_model_path:
        state_dict = torch.load(start_model_path)
        q_network.load_state_dict(state_dict)

    optimizer = optim.Adam(q_network.parameters(), lr=args.learning_rate)
    target_network = args.model(env.action_space.n).to(DEVICE)
    target_network.load_state_dict(q_network.state_dict())

    # Create replay buffer
    rb = TensorDictPrioritizedReplayBuffer(
        alpha=args.alpha,
        beta=args.beta,
        storage=LazyTensorStorage(args.buffer_size, device=DEVICE),
        batch_size=args.batch_size,
        prefetch=args.prefetch
    )

    obs = env.reset()
    obs = obs.float()

    # Tracks number of episodes simulated
    episode_cnt = 0
    # Data for each episode
    episode_scalars = []
    # Tracks the number of pieces we have played
    piece_count = 0
    # Whether we explore (play random moves) or exploit (play according to the model)
    explore = True
    info = None

    # Track the best scoring models
    episode_lines = deque(maxlen=args.mean_lines_count)
    best_mean_lines = -1.0

    eval_idx = 1

    if args.capture_inputs_video:
        out = cv2.VideoWriter(f'episode0.mp4', cv2.VideoWriter_fourcc(*'mp4v'), FPS, (BOARD_SHAPE[1]*SCALE_UP, BOARD_SHAPE[0]*SCALE_UP), False)

    sps_time = time.time()

    for global_step in range(args.total_timesteps):

        if global_step > 0 and global_step % 1000 == 0:
           curr_time = time.time()
           writer.add_scalar("charts/SPS", 1000 / (curr_time - sps_time), global_step)
           sps_time = curr_time

        if args.capture_inputs_video and (episode_cnt % args.video_frequency == 0):
            write_video_frame(out, obs, args.frame_stack, SCALE_UP)

        # If a new piece has been generated, decide wether we will explore or exploit for this piece
        if global_step > 0 and piece_count != info.get("piece_count"):
            piece_count = info.get("piece_count")
            if global_step < args.learning_starts:
                epsilon = args.start_e
            else:
                duration = args.exploration_fraction * (args.total_timesteps - args.learning_starts)
                epsilon = linear_schedule(args.start_e, args.end_e, duration, global_step - args.learning_starts)
            explore = (random.random() < epsilon)

        # Find the next action to play
        if explore:
            action = env.action_space.sample()
        else:
            q_values = q_network(obs)
            action = int(torch.argmax(q_values))

        # Play a step with the given action
        next_obs, reward, done, info = env.step(action)
        next_obs = next_obs.float()

        # Evaluate and report the agent periodically
        if trial and global_step > 0 and global_step % args.eval_frequency == 0 and global_step < (args.total_timesteps - args.total_evaluations):
            no_video = not (args.eval_video_frequency % eval_idx == 0)
            eval_mean_lines = evaluate(args, model=q_network, eval_name=f"{run_name}-eval-{eval_idx}", no_video=no_video)
            print(f"{prefix}evaluation_{eval_idx} mean_lines={eval_mean_lines}")
            trial.set_user_attr("mean_lines", eval_mean_lines)
            eval_idx += 1

            # Check if the trial should be pruned
            if trial and trial.should_prune():
                print(f"Pruning Run: {run_name}")
                raise optuna.exceptions.TrialPruned()

        if not done:
            # Add observation to replay buffer
            data = TensorDict({"obs" : obs,
                               "next_obs" : next_obs,
                               "action" : [action],
                               "reward" : [reward],
                               "done" : [int(done)]},
                               batch_size=1, device=DEVICE)
            rb.add(data)
            obs = next_obs
        else: # Episode completed!
            # Write episodic results
            episode_scalars.append((global_step, info, epsilon))
            if episode_cnt > 0 and (episode_cnt % 4) == (trial.number % 4):
                for global_step, info, epsilon in episode_scalars:
                    write_episode_scalars(writer, global_step, info, epsilon)
                episode_scalars = []

            episode_cnt += 1

            # If the model is outperfoming the best so far, make it the new best model
            episode_lines.append(info.get("lines"))
            if episode_cnt > args.mean_lines_count:
                curr_mean_lines = sum(episode_lines) / args.mean_lines_count
                if curr_mean_lines > best_mean_lines:
                    best_mean_lines = curr_mean_lines
                    if global_step > args.learning_starts:
                        print(f"{prefix}New best mean lines: {curr_mean_lines}")
                        # Keep a backup of the best scoring model
                        best_model_path = f"runs/{run_name}/{args.exp_name}.best"
                        torch.save(q_network.state_dict(), best_model_path)

            # Create new video file to write to
            if args.capture_inputs_video:
                if episode_cnt % args.video_frequency == 0:
                    out = cv2.VideoWriter(f'episode{episode_cnt}.mp4', cv2.VideoWriter_fourcc(*'mp4v'), FPS, (BOARD_SHAPE[1]*SCALE_UP, BOARD_SHAPE[0]*SCALE_UP), False)
                else:
                    out = None

            # Reload environment to avoid bugs within the emulator
            if episode_cnt % args.reload_env_frequency == 0:
                num_reloads = episode_cnt // args.reload_env_frequency
                del env
                env = get_env(args, run_name=f"{run_name}_{num_reloads}", capture_video=args.capture_video)

            obs = env.reset()

        # Training Logic
        if global_step > args.learning_starts:

            if global_step % args.train_frequency == 0:
                data = rb.sample()
                with torch.no_grad():
                    target_max, _ = target_network(torch.squeeze(data.get("next_obs"))).max(dim=1)
                    td_target = data.get("reward").flatten() + args.gamma * target_max * (1 - data.get("done").flatten())
                model_output = q_network(torch.squeeze(data.get("obs"))).gather(1, data.get("action")).squeeze()

                loss = F.mse_loss(model_output.float(), td_target.float())

                # optimize the model
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                # Update data priority
                td_error = torch.abs(model_output - td_target).unsqueeze(1)
                data.set("td_error", td_error)
                rb.update_tensordict_priority(data)

                # Log training statistics
                if global_step % (100 * args.train_frequency) == 0:
                    writer.add_scalar("losses/td_loss", loss, global_step)
                    writer.add_scalar("losses/q_values", model_output.mean().item(), global_step)

            # update target network
            if global_step % args.target_network_frequency == 0:
                for target_network_param, q_network_param in zip(target_network.parameters(), q_network.parameters()):
                    target_network_param.data.copy_(
                        args.tau * q_network_param.data + (1.0 - args.tau) * target_network_param.data
                    )

            if global_step % args.backup_frequency == 0:
                backup_model_path = f"runs/{run_name}/{args.exp_name}.backup"
                torch.save(q_network.state_dict(), backup_model_path)

    # End of main loop
    for global_step, info, epsilon in episode_scalars:
        # Write remaining episodic data
        write_episode_scalars(writer, global_step, info, epsilon)
    del episode_scalars

    # Save model
    final_model_path = f"runs/{run_name}/{args.exp_name}.final"
    torch.save(q_network.state_dict(), final_model_path)
    print(f"{prefix}model saved to {final_model_path}")

    # Evaluate model
    final_mean_lines = evaluate(
        args,
        model=q_network,
        eval_name=f"{run_name}-eval"
    )

    final_mean_lines = np.mean(final_mean_lines)
    print(f"{prefix}evaluation_{eval_idx}: mean_lines={final_mean_lines}")
    if trial:
        trial.set_user_attr("mean_lines", float(final_mean_lines))

  except:
    raise
  finally:
    env.close()
    writer.close()

# Optuna

In [25]:
# This class defines all training hyper-parameters
class Args:
    def __init__(self):
        # Optuna Settings
        self.num_trials = 100                   # Maximum number of trials
        self.num_jobs = 1                       # Number of trials to run in parallel
        self.num_startup_trials = 4             # Number of trials to run before random sampling is stopped

        # Settings
        self.exp_name = "Tetris_DQN"
        self.movement = SIMPLE_MOVEMENT
        self.run_id = int(time.time())
        self.torch_deterministic = True
        self.capture_video = False
        self.capture_eval_video = True
        self.capture_inputs_video = False
        self.backup_frequency = 50000
        self.mean_lines_count = 50
        self.video_frequency = 100
        self.eval_video_frequency = 5
        self.reload_env_frequency = 49
        self.prefetch = 3

        # Constant Hyper-Parameters
        self.seed = 1                           # Seed for all random sampling within a trial
        self.total_timesteps = 300_000          # Total number of time-steps run per trial
        self.buffer_size = 50_000               # Maximum size of the replay buffer
        self.learning_starts = 20_000           # How many time-steps pass before learning starts
        self.train_frequency = 1                # How often to train the model (1 = every time-step)
        self.batch_size = 32                    # Training batch-size
        self.gamma = 0.98                       # Cummulative discount multiplier
        self.tau = 0.999                        # Target network soft copy ratio
        self.target_network_frequency = 1000    # How often to update the target network
        self.exploration_fraction = 0.4         # What portion of the training to spend exploring
        self.frame_stack = 4                    # How many consecutive game-frames to stack when inputting to the model

        # Evaluation
        self.total_evaluations = 3              # Number of evaluations run over the course of a trial
        self.eval_episodes = 10                 # Number of episodes run when evaluating the model
        self.eval_frequency = self.total_timesteps // self.total_evaluations

        # Optimizable Hyper-Parameters
        self.model = None                       # What model to use (pretrained/empty)
        self.learning_rate = None               # Rate at which the NN is updated when training
        self.alpha = None                       # How much prioritization to use in the Replay-Buffer (0 = no priority, 1 = max)
        self.beta = None                        # How much priority-correction to use in our Replay-Buffer
        self.start_e = None                     # Exploration fraction at the beginning of training (higher)
        self.end_e = None                       # Exploration fraction at the end of the exploration fraction (lower)
        self.line_weight = None                 # Reward given from lines cleared
        self.height_weight = None               # Penalty given by board-height
        self.cost_weight = None                 # Penalty given by our custom cost penalty
        self.holes_weight = None                # Penalty given by number of holes in board
        self.bumpiness_weight = None            # Penalty given by bumpiness of board
        self.col_transitions_weight = None      # Penalty given by number of column transitions
        self.row_transitions_weight = None      # Penalty given by number of row transitions

args = Args()

In [26]:
# These are the hyper-parameters that Optuna will sample throughout the study
def sample_params(trial: optuna.Trial) -> dict:
    params = {
        # Learning hyper-parameters
        "model" : get_model_class(trial.suggest_categorical("model", ["FC", "Imitation1", "Imitation2", "Imitation3", "Imitation4", "Imitation5"])),
        "learning_rate" : trial.suggest_float("learning_rate", 1e-5, 1e-3, log=True),
        "alpha" : trial.suggest_float("alpha", 0, 1),
        "beta" : trial.suggest_float("beta", 0, 1),
        "start_e" : trial.suggest_float("start_e", 0.2, 1),
        "end_e" : trial.suggest_float("end_e", 0, 0.19),
        # Reward hyper-parameters
        "line_weight" : trial.suggest_float("line_weight", 0, 30),
        "height_weight" : trial.suggest_float("height_weight", 0, 3),
        "cost_weight" : trial.suggest_float("cost_weight", 0, 3),
        "holes_weight" : trial.suggest_float("holes_weight", 0, 3),
        "bumpiness_weight" : trial.suggest_float("bumpiness_weight", 0, 3),
        "col_transitions_weight" : trial.suggest_float("col_transitions_weight", 0, 3),
        "row_transitions_weight" : trial.suggest_float("row_transitions_weight", 0, 3),
    }
    return params

In [27]:
# The objective function: maximize mean number of lines cleared
def objective(trial: optuna.Trial):
    args = Args()
    hyperparameters = sample_params(trial)
    for key, value in hyperparameters.items():
        setattr(args, key, value)

    nan_encountered = False
    try:
      train(args, trial=trial)
    except AssertionError as e:
      # Sometimes, random hyperparams can generate NaN
      print(e)
      nan_encountered = True
    except optuna.exceptions.TrialPruned:
      raise

    # Tell the optimizer that the trial failed
    if nan_encountered:
        return float("nan")

    return trial.user_attrs["mean_lines"]

# Main

In [28]:
!rm -r runs/* videos/* *.mp4 db.sqlite3

rm: cannot remove 'videos/*': No such file or directory
rm: cannot remove '*.mp4': No such file or directory


In [29]:
study_name = f"{args.exp_name}_study"
study_num = 1

In [30]:
# Set pytorch num threads to 1 for faster training
torch.set_num_threads(1)
# Select the sampler, can be random, TPESampler, CMAES, ...
sampler = TPESampler(n_startup_trials=args.num_startup_trials)
# Pruner to stop bad runs
pruner = MedianPruner(n_startup_trials=args.num_startup_trials)
# Create the study and start the hyperparameter optimization
study = optuna.create_study(study_name=f"{study_name}-{study_num}", storage="sqlite:///db.sqlite3", sampler=sampler, pruner=pruner, direction="maximize")
study_num += 1

[I 2023-08-05 22:12:39,744] A new study created in RDB with name: Tetris_DQN_study-1


Run the following command in the terminal to open Optuna Dashboard:   
`optuna-dashboard sqlite:///db.sqlite3`

In [32]:
# Main training loop
# Interrupt this cell when you want to stop training
try:
    study.optimize(objective, n_trials=args.num_trials, n_jobs=args.num_jobs)
except KeyboardInterrupt:
    pass

In [ ]:
print(f"Number of finished trials: ", len(study.trials))
print(f"Best trial:")
best_trial = study.best_trial
print(f"\tValue: {best_trial.value}")
print(f"\tParams: ")
for key, value in best_trial.params.items():
    print(f"\t\t{key}: {value}")

print("\tUser attrs:")
for key, value in best_trial.user_attrs.items():
    print(f"\t\t{key}: {value}")

In [ ]:
# Write report
study.trials_dataframe().to_csv(f"study_results_dqn_{study_name}_{study_num}.csv")

fig1 = plot_optimization_history(study)
fig2 = plot_param_importances(study)

fig1.show()
fig2.show()